# Example 2: Function fitting
## SI Section 2.4: Regression of a deterministic physical function
### Reference: INN paper https://arxiv.org/abs/2404.10296

In this example, we fit a 10-input 1-output function described in the SI Section 2.4 of the INN paper.

From this function, 1,000,000 randomly sampled data using Latin hypercube sampling are generated and divided into 70% for training, 15% for validation, and 15% for testing. 

The first step is to import the pyinn package.

In [8]:
from jax import config
import jax.numpy as jnp
config.update("jax_enable_x64", True)
import os, sys
import yaml
import importlib

from pyinn import dataset_classification, dataset_regression, model, train, plot # with pyinn library


First, set up GPUs

In [9]:
gpu_idx = 2  # set which GPU to run on Athena
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # GPU indexing
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_idx)  # GPU indexing

Next, we create the dataset and store it in /data directory in the current folder. The created datafile will be named as data_name = "10D_5D_physics.csv"

In [10]:
# Define configuration of the dataset & training
data_name = '10D_5D_physics' # data file name in /data directory
config = {}
config["DATA_PARAM"] = {
    "data_name": data_name,  # name of the dataset
    "input_col": [0,1,2,3,4,5,6,7,8,9],  # input columns in the dataset
    "output_col": [10,11,12,13,14], # output columns in the dataset

    "bool_data_generation": True,  # data is already stored and splitted
    "data_size": 1000000,
    "split_ratio": [0.7, 0.15, 0.15],  # [0.7,0.15,0.15]

    "bool_normalize": True,  # whether we normalize the input data or not
    "bool_shuffle": True  # whether we shuffle the data
}
config["TRAIN_PARAM"] = {
    "num_epochs_INN": 100,
    "num_epochs_MLP": 100,
    "batch_size": 128,
    "learning_rate": 1e-3,
    "bool_train_acc": False,

    "validation_period": 10,
    "bool_denormalize": False,  # or True, whether we denormalize when measuring errors
    "error_type": "mse",  # rmse or mse
    "patience": 10,
    "stopping_loss_train": 4e-4
}

# Create dataset and save it in /data directory
data = dataset_regression.Data_regression(data_name, config)
  

loaded 1000000 datapoints from 10D_5D_physics dataset


Now the datafile was created at /data/1D_1D_sine_10000.csv. The file name ending with "_10000" denotes the data_size in the config variable. Let us visualize the data using pandas.

In [11]:
import pandas as pd

# Load the CSV file
data_size = int(config["DATA_PARAM"]["data_size"])
df = pd.read_csv(f'./data/{data_name}_{data_size}.csv')

# Display the first few rows
df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,u1,u2,u3,u4,u5
0,0.385983,0.756249,0.397685,0.609083,0.122860,0.667948,0.210607,0.224966,0.696133,0.338887,-197.355468,0.358267,6.097321,0.773473,248.568777
1,0.830538,0.946635,0.481156,0.265116,0.719175,0.168365,0.658874,0.987596,0.123958,0.805102,-403.578452,0.348198,5.609222,0.694016,302.451919
2,0.422028,0.314211,0.286128,0.192950,0.257675,0.524438,0.605413,0.905532,0.414083,0.983750,-90.530637,0.470854,4.940361,0.771587,277.354997
3,0.316487,0.774567,0.120525,0.880524,0.103807,0.631655,0.652926,0.195743,0.969031,0.517211,-281.252093,0.264405,6.071011,1.428927,209.645109
4,0.978135,0.816018,0.683751,0.723170,0.648270,0.941244,0.467634,0.426148,0.440896,0.861603,-276.189190,0.436724,5.043639,0.770566,308.603082


The datafile contains 15 columns: x1 ~ x10, u1 ~ u5.
Next, we define INN hyperparameters. According to Figure 4(a) of the INN paper, the following hyperparameters were used.

* nmode: number of modes; integer

nnmode: 14

* nseg: number of segments for each dimension; integer or a list of integers

nseg: 10

* s_patch; integer, 0 <= s_patch < nseg/2

s_patch: 2

* INN activation; string

INN activation: polynomial

* p_order; integer, p_order <= s_patch

p_order: 2

In [12]:
config["MODEL_PARAM"] = {
    "nmode": 14,
    "nseg": 10,
    "s_patch": 2,
    "INNactivation": "polynomial", # INN activation function, can be 'polynomial', 'sinusoidal', 'exponential', 'sigmoid', 'tanh', 'gelu'
    "p_order": 2,
    ### non-adjustable parameters
    "radial_basis": "cubicSpline",
    "alpha_dil": 20 # this is a dilation parameter and will be set as 20 for default.
}

Let's finalize the configuration file.

In [13]:
if config["MODEL_PARAM"]["s_patch"] > 0:
    config['interp_method'] = "nonlinear" # nonlinear (Q=2 INN message passing) interpolation
elif config["MODEL_PARAM"]["s_patch"] == 0:
    config['interp_method'] = "linear" # linear (Q=1 INN message passing) interpolation
config['TD_type'] = "CP"

Train INN (Q=2 nonlinear interpolation)

In [14]:
regressor = train.Regression_INN(data, config)  
regressor.train()  # Train module
params = regressor.params
errors_train, errors_val, errors_epoch = regressor.errors_train, regressor.errors_val, regressor.errors_epoch, 


edex_max / ndex_max: 6 / 5
------------ INN CP nonlinear, nmode: 14, nseg: 10, s=2, P=2 -------------
# of training parameters: 7700
Epoch 1
	Train mse: 9.4849e-03
	48.62 seconds per epoch
Epoch 2
	Train mse: 4.0226e-04
	49.72 seconds per epoch
Epoch 3
	Train mse: 2.1738e-04
	49.58 seconds per epoch
Training took 148.7513 seconds/ 49.58 seconds per epoch
Test
	Test MSE: 1.9039e-04
	Test took 9.9480 seconds
	Inference time: 0.0001 seconds
